In [44]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

url = "https://montanafreepress.org/"
resp = requests.get(url, timeout=20, headers={"User-Agent": "Mozilla/5.0"})
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")

articles = soup.select("article")  # broader match
print(f"Found {len(articles)} article elements")

def pick_text(el):
    return el.get_text(strip=True) if el else None

def parse_article(art):
    # Title/link: try multiple selectors in order of reliability
    a = (art.select_one("a[rel='bookmark']") or
         art.select_one("h2.entry-title a") or
         art.select_one("h3.entry-title a") or
         art.select_one("header .entry-title a"))
    title = pick_text(a)
    link  = a["href"] if a and a.has_attr("href") else None

    # Excerpt
    p = (art.select_one(".entry-wrapper > p") or
         art.select_one("p"))
    excerpt = pick_text(p)

    # Category
    cat = art.select_one(".cat-links a")
    category = pick_text(cat)

    # Author
    auth = (art.select_one(".byline .author a") or
            art.select_one(".byline a"))
    author = pick_text(auth)

    # Published/updated times
    pub = art.select_one("time.published, time.entry-date")
    upd = art.select_one("time.updated")
    published_iso = pub.get("datetime") if pub else None
    updated_iso = upd.get("datetime") if upd else None

    # Image
    img = art.select_one("figure.post-thumbnail img, img.wp-post-image")
    image_src = (img.get("src") or img.get("data-src") or img.get("data-lazy-src")) if img else None
    image_alt = img.get("alt") if img else None

    return {
        "title": title, "url": link, "excerpt": excerpt,
        "category": category, "author": author,
        "published_iso": published_iso, "updated_iso": updated_iso,
        "image_src": image_src, "image_alt": image_alt
    }

data = [parse_article(a) for a in articles]

# Preview
for d in data:
    print("Title if Found:", d["title"], "URL:", d["url"], "Excerpt if found:", d["excerpt"])

with open("articles.txt", "a", encoding="utf-8") as f:
    for d in data:
        if d.get("title") is not None:
            f.write("Title: " + d["title"])
        if d.get("url") is not None:
            f.write(" URL: " + d["url"])
        if d.get("excerpt") is not None:
            f.write(" Excerpt if found: " + d["excerpt"])
        f.write("\n\n")

Found 31 article elements
Title if Found:  URL: https://montanafreepress.org/2025/10/23/democratic-candidates-emphasize-ranch-roots-and-military-experience-in-montanas-western-house-primary/ Excerpt if found: An investigation into longtime Helena oncologist Thomas Weiner sparked concerns about past medical care. But Montana’s routes to accountability are often stacked with obstacles.
Title if Found:  URL: https://montanafreepress.org/2025/10/23/democratic-candidates-emphasize-ranch-roots-and-military-experience-in-montanas-western-house-primary/ Excerpt if found: None
Title if Found:  URL: https://montanafreepress.org/2025/10/20/railroad-giant-seeks-to-overturn-montana-asbestos-liability-verdict/ Excerpt if found: None
Title if Found:  URL: https://montanafreepress.org/2025/10/23/west-yellowstone-mayor-resigns-after-social-media-comments/ Excerpt if found: None
Title if Found: Legislator Bill Mercer confirmed to U.S. District Court judgeship URL: https://montanafreepress.org/2025/10/23